In [2]:
import os
import librosa
import numpy as np
# import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import display
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm
from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features_from_waveform
### OPTIMIZE IMPORT ###

In [3]:
def extract_features(waveforms, components_list, statistics_list=None):
    """This is an important function. Given a list of Waveform objects, a list of
    Waveform methods in the form of strings and a list of Barrel methods in the
    form of strings, compute the time-independent features resulting. This function
    does multiprocessing.

    Args:
        waveforms (list of Waveform): This is a list of waveform objects
        components_list (list of str/dict): This is a list of the methods which
            should be applied to all the waveform objects in waveforms. If a dict,
            this also contains arguments to the sound.Waveform methods.
        statistics_list (list of str): This is a list of the methods which
            should be applied to all the time-dependent features computed
            from the waveforms.

    Returns:
        pandas DataFrame: pandas dataframe where every row corresponds
            to features extracted for one of the waveforms and columns
            represent individual features.
    """
    output_feats = []
    # waveforms = tqdm(waveforms, desc='Extracting features...')

    for wave in waveforms:
        output_feats.append(
            extract_features_from_waveform(
                components_list, statistics_list, wave
            )
        )

    return pd.DataFrame(output_feats)

Используемые библиотеки:
1) liborsa
2) surfboard
3)

Порядок действий:
    Усреднить громкость наверное
    1) librosa.effects.preemphasis (In order to equalize the effect of the propagation)
    2) features:
        2.1)MFCC
        2.2)Energy
        2.3)Pitch
        2.4)formants (1 - 3)
        2.5)spectrum centroid????
        2.6)Zero crossing rate
        2.7)jitter
        2.8)shimmers
        2.7) Mean, max, std (статистика по всем фичам на запись)

In [31]:
# sound_path = "../data/1/03-01-05-01-01-01-01_A1.wav"
# components_list_for_plain_features = [
#         'f0_contour', 'log_energy', 'spectral_centroid', 'loudness', 'intensity', 'rms', 'ppe']
# mfcc_with_arg = {'mfcc': {'n_mfcc': 16, 'n_fft_seconds': 512 / 48000}}
# statistics_list = ['mean', 'std', 'min', 'max', 'skewness', 'kurtosis']
# components_list_for_plain_features.append(mfcc_with_arg)
#
# audio, sampling_freq = librosa.load(sound_path, sr=None, res_type='scipy')
# # 512 / sampling_freq
# processed_audio = librosa.effects.preemphasis(audio)
# # Extract dataframe...
# audio = Waveform(signal=processed_audio, sample_rate=sampling_freq)
# feature_df2 : pd.DataFrame= extract_features(
#     waveforms=[audio], components_list=components_list_for_plain_features)
# # display(feature_df2.mfcc[0][2:, ].ravel().shape)
# feature_df2 = unpack_features_inplace(feature_df2)
# feature_df2

23.967546906279438

10165.480300419653

8.333792e-06

0.0014897384

-2.138103



C:\Users\Admin\.conda\envs\Kaggle_s\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


ValueError: Shape of passed values is (1, 5082), indices imply (1, 1)

In [20]:
components_list_for_stat_features = [
    'f0_contour', 'log_energy', 'spectral_centroid', 'formants', 'loudness',
    'jitters', 'shimmers', 'zerocrossing',
]
components_list_for_plain_features = [
        'f0_contour', 'log_energy', 'spectral_centroid', 'loudness', 'intensity', 'rms', 'ppe']
mfcc_with_arg = {'mfcc': {'n_mfcc': 16, 'n_fft_seconds': 512 / 48000}}
statistics_list = ['mean', 'std', 'min', 'max', 'skewness', 'kurtosis']
components_list_for_stat_features.append(mfcc_with_arg)
components_list_for_plain_features.append(mfcc_with_arg)

def unpack_features_inplace(feature_dataframe : pd.DataFrame) -> pd.DataFrame:
    result  = pd.DataFrame()
    for column in feature_dataframe.columns:
        if type(feature_dataframe[column][0]) is np.ndarray:
            if column is "mfcc":
                # get 2 - 16 mfcc coef for each window
                features_arr : np.ndarray = feature_dataframe.mfcc[0][2:, 50:250].ravel()
                features_arr = features_arr[np.newaxis]
                length = features_arr.shape[1]
            else:
                features_arr = feature_dataframe[column][0][:, 50:250]
                length = len(features_arr[0])

            print(features_arr.shape)
            df = pd.DataFrame(features_arr, columns=[column + str(i) for i in range(length)])
            result = pd.concat([result, df], axis=1)
        else:
            result = pd.concat([result, feature_dataframe[column]], axis=1)
    return result

def get_features(sound_path : str) -> pd.DataFrame:
    audio, sampling_freq = librosa.load(sound_path, sr=None, res_type='scipy')
    # 512 / sampling_freq
    processed_audio = librosa.effects.preemphasis(audio)
    # Extract dataframe...
    audio = Waveform(signal=processed_audio, sample_rate=sampling_freq)
    feature_df1 = extract_features(
        waveforms=[audio], components_list=components_list_for_stat_features, statistics_list=statistics_list)
    feature_df2 : pd.DataFrame= extract_features(
        waveforms=[audio], components_list=components_list_for_plain_features)
    feature_df2 = unpack_features_inplace(feature_df2)
    feature_df = pd.concat([feature_df1, feature_df2], axis=1)
    return feature_df


In [21]:
global_audio_path = "../data"
res_dataframe = pd.DataFrame()
classes = []
for i in range(1, 6):
    for root, dirs, files in os.walk(os.path.join(global_audio_path, str(i)), topdown=False):
        print(len(files))
        for name in files:
            important_features = get_features(os.path.join(root, name))
            # display(important_features)
            res_dataframe = res_dataframe.append(important_features, ignore_index=True)
            classes.append(i)

172
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1, 200)
(1, 2800)
(1, 200)
(1, 200)
(1, 200)
(1,

C:\Users\Admin\.conda\envs\Kaggle_s\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
C:\Users\Admin\.conda\envs\Kaggle_s\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
C:\Users\Admin\.conda\envs\Kaggle_s\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
C:\Users\Admin\.conda\envs\Kaggle_s\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filter

In [22]:
display(res_dataframe)
res_dataframe.describe()

,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,log_energy,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_min,...,mfcc2790,mfcc2791,mfcc2792,mfcc2793,mfcc2794,mfcc2795,mfcc2796,mfcc2797,mfcc2798,mfcc2799
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,-50.789417,10165.480300,4205.444653,0.000000,...,-5.784768,6.941507,9.789982,-0.183497,7.684388,2.465991,16.278410,14.213696,9.772486,14.578217
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-52.897926,8238.646944,4965.301582,0.000000,...,-24.181034,-11.925888,-2.965387,-6.677677,-20.976917,-2.799877,-3.983200,-10.706288,2.820465,1.374095
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,-57.779049,10573.577648,2372.872379,5207.613500,...,7.848310,7.157007,8.775260,14.798556,4.345983,1.526945,-1.860492,4.149419,-0.724602,-4.088886
3,28.523561,74.396412,0.0,260.546090,2.255366,3.177505,-50.261565,8503.507631,5009.949377,0.000000,...,25.945702,15.956472,12.133972,12.979851,8.674249,13.092389,13.643843,8.264226,17.293447,9.562870
4,10.253963,40.387550,0.0,238.921752,3.916850,14.445161,-46.754228,10341.238850,4201.536338,3761.336784,...,2.131070,6.721894,-6.000320,6.073660,5.282675,4.403518,-5.214548,3.379026,8.097569,3.939929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,-49.614353,8274.023605,4547.170923,0.000000,...,-11.479608,-7.673368,-22.498039,-12.937588,-12.490738,-26.901779,-19.626822,-19.390402,-20.214138,-15.030752
855,11.937603,40.763433,0.0,176.104431,3.170061,8.235236,-50.706402,9779.106757,4538.508236,3203.973249,...,5.654816,8.972028,-3.383322,8.414681,-10.405883,5.349158,3.556877,-4.340685,-6.716612,9.199011
856,80.651854,98.998965,0.0,237.845414,0.422148,-1.802225,-55.082888,10840.344698,3517.150012,5156.937923,...,11.688147,15.075483,11.824942,10.563176,11.821394,13.225666,13.227609,13.606882,10.666561,15.671804
857,47.841428,76.043443,0.0,200.364790,0.980022,-0.996395,-55.624970,10787.963161,2634.707493,6508.520967,...,1.256448,13.222193,5.746243,5.726025,-4.034789,4.713089,4.652605,13.168962,1.757402,7.999271


,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,log_energy,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_min,...,mfcc2790,mfcc2791,mfcc2792,mfcc2793,mfcc2794,mfcc2795,mfcc2796,mfcc2797,mfcc2798,mfcc2799
count,859.000000,859.000000,859.0,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,...,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000
mean,32.679361,61.670470,0.0,204.244583,2.089628,8.862409,-55.532670,10279.629365,3908.153515,2562.174655,...,2.401791,2.339218,2.338738,2.411586,2.584620,2.609919,2.607384,3.072637,2.558111,2.414633
std,20.745549,26.378037,0.0,66.311670,2.529656,42.154758,9.849781,1128.374344,754.819620,2282.487767,...,9.854339,9.777829,9.854805,9.272508,9.251390,9.652595,9.181265,9.123116,8.845971,8.523480
min,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,-79.909811,6901.756837,1637.548757,0.000000,...,-31.232822,-29.605165,-34.033295,-30.158756,-31.045221,-26.935404,-28.733269,-28.984810,-31.691265,-29.617165
25%,17.189673,45.091285,0.0,180.123285,0.985625,-0.941569,-62.464710,9504.400733,3377.998034,0.000000,...,-3.499070,-2.868422,-2.632383,-2.740249,-2.222687,-2.600809,-2.468223,-2.312024,-2.300217,-1.963377
50%,30.608498,63.876307,0.0,226.122932,1.483075,0.337269,-57.157595,10460.723355,3905.091578,3321.681608,...,1.473361,1.400232,1.591738,1.650529,1.696271,1.299231,1.532429,1.853444,1.757402,1.542664
75%,47.769585,83.996671,0.0,251.080904,2.181621,3.039097,-48.948679,11137.980792,4416.606774,4338.295963,...,7.604359,7.253968,7.630193,7.267267,7.097432,7.320981,7.495709,8.673398,7.239559,6.934378
max,96.018157,109.769293,0.0,280.559255,21.166063,446.002222,-29.154129,12816.873023,5657.350033,7484.228202,...,38.839424,38.130211,41.478107,38.894073,36.155029,50.561028,42.553017,34.411968,34.715691,36.445576


In [24]:
res_dataframe.fillna(0, inplace=True)
target = pd.Series(classes, name="target")
target.unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [33]:
sum_df = res_dataframe.sum(axis=0)
index_to_del = sum_df[sum_df == 0].index
print(index_to_del)
sum_df

Int64Index([6735, 6736, 6737, 6738, 6739, 6740, 6741, 6742, 6743, 6744, 6745,
            6746, 6747, 6748, 6749],
           dtype='int64')


0        471.002571
1       1469.362606
2       1779.936194
3       1624.555274
4       1206.148421
           ...     
6760       0.704981
6761       0.664472
6762       0.621463
6763       0.576115
6764       0.528595
Length: 6765, dtype: float64

In [4]:
# load old data
data = pd.read_csv("features_and_stat_full.csv")
target = data["target"]
res_dataframe = data.drop(columns="target")
display(res_dataframe)
display(target)

,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,log_energy,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_min,...,mfcc_kurtosis_7,mfcc_kurtosis_8,mfcc_kurtosis_9,mfcc_kurtosis_10,mfcc_kurtosis_11,mfcc_kurtosis_12,mfcc_kurtosis_13,mfcc_kurtosis_14,mfcc_kurtosis_15,mfcc_kurtosis_16
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,-50.789417,10165.480300,4205.444653,0.000000,...,0.576864,1.707728,0.463653,1.532729,0.032541,1.781521,1.504384,1.459517,2.456224,1.134118
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-52.897926,8238.646944,4965.301582,0.000000,...,0.379027,4.740769,1.378276,2.137689,2.199326,2.473318,1.252770,2.492139,1.871985,1.556884
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,-57.779049,10573.577648,2372.872379,5207.613500,...,0.023238,0.361595,-0.043995,1.784541,1.154673,0.677837,0.467001,0.377071,1.246487,0.543016
3,28.523561,74.396412,0.0,260.546090,2.255366,3.177505,-50.261565,8503.507631,5009.949377,0.000000,...,0.812738,1.566179,0.281920,0.789175,2.307541,3.260491,0.920348,2.363914,0.982231,0.563017
4,10.253963,40.387550,0.0,238.921752,3.916850,14.445161,-46.754228,10341.238850,4201.536338,3761.336784,...,-0.133761,2.109509,0.715887,1.868507,0.366396,1.362177,1.461011,1.836645,2.821719,0.674766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,-49.614353,8274.023605,4547.170923,0.000000,...,-0.947450,0.660020,0.277131,-0.148228,0.011930,0.953463,0.417874,1.400286,2.256507,2.187404
855,11.937603,40.763433,0.0,176.104431,3.170061,8.235236,-50.706402,9779.106757,4538.508236,3203.973249,...,0.509558,0.890578,1.312091,0.770454,0.157578,3.085950,1.037611,0.771993,0.071554,0.999199
856,80.651854,98.998965,0.0,237.845414,0.422148,-1.802225,-55.082888,10840.344698,3517.150012,5156.937923,...,1.103626,1.679635,0.548185,0.956269,0.935985,2.107008,2.106866,0.296404,1.902266,2.004494
857,47.841428,76.043443,0.0,200.364790,0.980022,-0.996395,-55.624970,10787.963161,2634.707493,6508.520967,...,1.064522,0.811155,-0.365453,0.695748,0.256684,0.911476,1.227208,0.329263,0.871274,0.810664


0      1
1      1
2      1
3      1
4      1
      ..
854    5
855    5
856    5
857    5
858    5
Name: target, Length: 859, dtype: int64

Секция с Feature selection

In [25]:
# feature names to drop (based on MI decomposition method)
res = mutual_info_classif(res_dataframe, target, random_state=42, n_neighbors=30)
df_res = pd.Series(res)
sorted_df = df_res.sort_values(ascending=False)
index_to_del : pd.Int64Index = sorted_df[sorted_df < 0.1].index
print(index_to_del.shape)
columns_to_del = [res_dataframe.columns[i] for i in index_to_del]
for c in columns_to_del:
    print(c)
# features_cleared_df = res_dataframe.drop(columns=)
# features_cleared_df

(3243,)
rms24
mfcc195
spectral_centroid194
mfcc197
mfcc66
mfcc115
intensity40
spectral_centroid32
rms40
intensity30
intensity26
mfcc67
f0_contour136
f0_contour131
intensity33
rms29
f0_contour142
f0_contour137
apq3Shimmer
spectral_centroid198
mfcc151
spectral_centroid21
intensity27
mfcc158
spectral_centroid196
spectral_centroid197
f0_contour67
spectral_centroid193
f0_contour139
intensity28
mfcc_mean_13
mfcc36
f0_contour126
f0_contour66
spectral_centroid22
mfcc113
f0_contour55
mfcc160
mfcc159
spectral_centroid_skewness
f0_contour129
localShimmer
mfcc32
f0_contour138
f0_contour151
intensity24
rms25
spectral_centroid171
f0_contour62
spectral_centroid165
mfcc35
spectral_centroid170
mfcc114
spectral_centroid161
spectral_centroid184
rms30
intensity25
mfcc72
f0_contour134
mfcc34
f0_contour64
f0_contour56
mfcc153
mfcc20
f0_contour133
mfcc53
mfcc_kurtosis_5
mfcc157
rms23
spectral_centroid31
f0_contour125
spectral_centroid162
mfcc148
mfcc155
mfcc154
mfcc22
mfcc21
spectral_centroid38
mfcc156
spect

In [5]:
# k = 90 for 1st set
# k = 200
features_cleared_df = SelectKBest(mutual_info_classif, k=90).fit_transform(res_dataframe, target)
pd.DataFrame(features_cleared_df)

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,-50.789417,10165.480300,4205.444653,0.000000,...,3.794117,-1.108763,0.732241,0.480405,0.576864,0.032541,1.781521,1.504384,2.456224,1.134118
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-52.897926,8238.646944,4965.301582,0.000000,...,1.741925,-0.398524,1.462646,1.335705,0.379027,2.199326,2.473318,1.252770,1.871985,1.556884
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,-57.779049,10573.577648,2372.872379,5207.613500,...,4.559898,-0.304439,0.392138,0.636229,0.023238,1.154673,0.677837,0.467001,1.246487,0.543016
3,28.523561,74.396412,0.0,260.546090,2.255366,3.177505,-50.261565,8503.507631,5009.949377,0.000000,...,4.336523,-0.102940,2.108052,1.666078,0.812738,2.307541,3.260491,0.920348,0.982231,0.563017
4,10.253963,40.387550,0.0,238.921752,3.916850,14.445161,-46.754228,10341.238850,4201.536338,3761.336784,...,4.282357,-0.916790,1.306719,-0.327491,-0.133761,0.366396,1.362177,1.461011,2.821719,0.674766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,-49.614353,8274.023605,4547.170923,0.000000,...,2.616873,-1.026287,1.258226,1.567229,-0.947450,0.011930,0.953463,0.417874,2.256507,2.187404
855,11.937603,40.763433,0.0,176.104431,3.170061,8.235236,-50.706402,9779.106757,4538.508236,3203.973249,...,2.177329,-1.168689,2.328687,-0.599227,0.509558,0.157578,3.085950,1.037611,0.071554,0.999199
856,80.651854,98.998965,0.0,237.845414,0.422148,-1.802225,-55.082888,10840.344698,3517.150012,5156.937923,...,3.644388,-0.505122,0.845884,-0.872183,1.103626,0.935985,2.107008,2.106866,1.902266,2.004494
857,47.841428,76.043443,0.0,200.364790,0.980022,-0.996395,-55.624970,10787.963161,2634.707493,6508.520967,...,3.855180,-1.060770,2.889217,-0.277684,1.064522,0.256684,0.911476,1.227208,0.871274,0.810664


Секция с классификаторами

In [ ]:
X = scale(res_dataframe)
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
params = {"max_depth" : [3,5,7], "n_estimators" : [100, 200, 400]}
print(params)
estimator_ = GradientBoostingClassifier()
gs = GridSearchCV(estimator_, params, cv=cv,scoring="accuracy", verbose=2, n_jobs=-1)
gs.fit(X, target)
gs.cv_results_

In [70]:
X = scale(res_dataframe)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = KNeighborsClassifier(n_neighbors=11)
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1)
# best ~ 0.56
scores.mean()

0.5646198830409357

In [33]:
# X = scale(res_dataframe)
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = LinearSVC(C=0.01, dual=False)
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1)
# best ~ 0.64
scores.mean()

0.5413504691962464

In [32]:
X = scale(res_dataframe)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = LogisticRegression("l2", C=0.001, solver="sag")
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
# best ~ 0.58
scores.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    5.6s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.7s finished


0.6100435196518428

In [85]:
X = res_dataframe
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = DecisionTreeClassifier(splitter="random")
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
# best ~ 0.58
scores.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


0.4435060519515844

In [81]:
X = scale(res_dataframe)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = GradientBoostingClassifier()
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
scores.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    9.9s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   10.0s finished


0.6147558819529444

In [8]:
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = MLPClassifier(hidden_layer_sizes=100, learning_rate='adaptive', solver='adam')
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
scores.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    2.8s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.8s finished


0.6403644770841833

In [10]:
X = scale(res_dataframe)
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
params = {"C" : 10 ** np.arange(-3, 3, dtype = float), "degree" : [1, 2, 5, 9]}
print(params)
estimator_ = SVC(kernel="poly")
gs = GridSearchCV(estimator_, params, cv=cv,scoring="accuracy", verbose=2, n_jobs=-1)
gs.fit(X, target)
gs.cv_results_

{'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'degree': [1, 2, 5, 9]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    3.7s finished


{'mean_fit_time': array([0.09148345, 0.099089  , 0.10509639, 0.09848862, 0.09108243,
        0.0966877 , 0.09588294, 0.11230145, 0.07626882, 0.09068227,
        0.09908924, 0.12010942, 0.06285706, 0.09848914, 0.10369301,
        0.11750598, 0.0738667 , 0.11070065, 0.11930799, 0.12831597,
        0.14553185, 0.11490469, 0.11931219, 0.11951218]),
 'std_fit_time': array([0.00463449, 0.00597178, 0.00522054, 0.00403344, 0.00800818,
        0.00680609, 0.00402167, 0.01702551, 0.00519699, 0.00377678,
        0.00569762, 0.00879469, 0.00458242, 0.01197118, 0.00224734,
        0.00771666, 0.01065641, 0.00944453, 0.01179876, 0.01593197,
        0.01501482, 0.00714488, 0.00896119, 0.00149845]),
 'mean_score_time': array([0.01541481, 0.01741581, 0.01481185, 0.01581411, 0.01481299,
        0.01441293, 0.01461334, 0.01601419, 0.0136127 , 0.01401286,
        0.01681523, 0.01681533, 0.01541395, 0.02322149, 0.01581535,
        0.01561456, 0.01261144, 0.01441336, 0.01561465, 0.01521392,
        0.011610

In [34]:
X = scale(res_dataframe)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = RandomForestClassifier(n_estimators=500)
scores : np.ndarray = cross_val_score(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
scores.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   11.6s remaining:    7.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.2s finished


0.5425472596219231

In [7]:
# GMM experience
gm = GaussianMixture()
gm.fit(scale(features_cleared_df))
gm.means_

array([[ 2.48152294e-16, -1.81978349e-16,  0.00000000e+00,
         2.48152294e-16, -7.03098167e-17, -1.24076147e-17,
        -1.99555803e-16, -2.48152294e-17,  3.47413212e-16,
         2.06793579e-16, -1.42687569e-16,  4.15655093e-16,
         6.61739451e-17, -1.20715751e-16, -3.30869726e-16,
        -1.65434863e-16,  2.23337065e-16, -9.51250461e-17,
        -5.99701378e-17,  1.15804404e-16, -3.66024634e-16,
        -1.15804404e-16, -1.15804404e-16, -7.03098167e-17,
        -4.27545724e-16,  2.89511010e-17, -1.24076147e-17,
        -5.16983946e-17, -1.90250092e-16,  1.81978349e-16,
        -2.89511010e-17,  2.48152294e-17,  2.48152294e-17,
        -3.10190368e-17, -9.09891745e-17,  3.72228441e-17,
         1.34415826e-16,  2.68831652e-16,  4.96821572e-16,
        -7.44456883e-17,  4.96304588e-17,  4.71489359e-16,
         1.81978349e-16, -1.48891377e-16, -9.97779016e-17,
        -2.87960058e-16, -5.04576332e-16,  2.23337065e-16,
        -6.70011194e-16,  1.98521835e-16,  4.65285552e-1

In [6]:
estimators = [
    ('mkp', MLPClassifier(hidden_layer_sizes=100, learning_rate='adaptive', solver='adam')),
    ('svc', LinearSVC(C=0.01, dual=False)),
    ("rf", RandomForestClassifier(n_estimators=500))
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
scores : np.ndarray = cross_val_score(clf, X, target, cv = cv_, n_jobs=-1, verbose=2)
# score ~ 0.78
print("Min - " , np.min(scores))
print("Max - " , np.max(scores))
print("Mean - " , scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   20.6s remaining:   13.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.9s finished


Min -  0.7441860465116279
Max -  0.8187134502923976
Mean -  0.7788589691282469


In [11]:
gs.best_params_


{'C': 1.0, 'degree': 1}

In [26]:
full_df = pd.concat([res_dataframe, target], 1)
full_df.to_csv("features_and_stat_full_with_plain_.csv", index=False)

